<h2>Capture Images</h2>


In [2]:
import cv2
import os


def capture_image(student_name, num_images=50):
    # Create folder for the student if it doesn't exist
    if not os.path.exists(f"dataset/{student_name}"):
        os.makedirs(f"dataset/{student_name}")

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Capture", cv2.WINDOW_NORMAL)

    print(
        f"Press 'C' to capture {num_images} images for {student_name} or 'Q' to quit..."
    )

    count = 0
    while count < num_images:
        ret, frame = cap.read()  # Read frame from the webcam
        if ret:
            cv2.imshow("Capture", frame)  # Show live feed

            key = cv2.waitKey(1) & 0xFF  # Capture key press

            if key == ord("c"):  # Press 'C' to capture image
                image_path = f"dataset/{student_name}/image{count+1}.jpg"
                cv2.imwrite(image_path, frame)  # Save the image
                print(f"Captured image {count+1} for {student_name}: {image_path}")
                count += 1

            elif key == ord("q"):  # Press 'Q' to quit capturing
                print("Quitting image capture.")
                break

    # Release webcam and destroy windows
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


# Add new student names in the array
studentNames = ["Sarim Toqeer"]
for studentName in studentNames:
    capture_image(studentName, num_images=50)

Press 'C' to capture 50 images for Sarim Toqeer or 'Q' to quit...
Captured image 1 for Sarim Toqeer: dataset/Sarim Toqeer/image1.jpg
Captured image 2 for Sarim Toqeer: dataset/Sarim Toqeer/image2.jpg
Captured image 3 for Sarim Toqeer: dataset/Sarim Toqeer/image3.jpg
Captured image 4 for Sarim Toqeer: dataset/Sarim Toqeer/image4.jpg
Captured image 5 for Sarim Toqeer: dataset/Sarim Toqeer/image5.jpg
Captured image 6 for Sarim Toqeer: dataset/Sarim Toqeer/image6.jpg
Captured image 7 for Sarim Toqeer: dataset/Sarim Toqeer/image7.jpg
Captured image 8 for Sarim Toqeer: dataset/Sarim Toqeer/image8.jpg
Captured image 9 for Sarim Toqeer: dataset/Sarim Toqeer/image9.jpg
Captured image 10 for Sarim Toqeer: dataset/Sarim Toqeer/image10.jpg
Captured image 11 for Sarim Toqeer: dataset/Sarim Toqeer/image11.jpg
Captured image 12 for Sarim Toqeer: dataset/Sarim Toqeer/image12.jpg
Captured image 13 for Sarim Toqeer: dataset/Sarim Toqeer/image13.jpg
Captured image 14 for Sarim Toqeer: dataset/Sarim Toqee

<h1>Dataset Processing</h1>


In [ ]:
import os
import random
import numpy as np
import cv2
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf


# Preprocess function
def preprocess_image(image_path, image_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, image_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    return img


# Create image pairs function
def create_image_pairs(image_dir):
    students = os.listdir(image_dir)
    pairs = []
    labels = []

    # Generate positive pairs (same student)
    for student in students:
        student_path = os.path.join(image_dir, student)
        if not os.path.isdir(student_path):
            continue
        student_images = os.listdir(student_path)
        for i in range(len(student_images)):
            for j in range(i + 1, len(student_images)):
                img1 = f"{image_dir}/{student}/{student_images[i]}"
                img2 = f"{image_dir}/{student}/{student_images[j]}"
                pairs.append([preprocess_image(img1), preprocess_image(img2)])
                labels.append(1)

    # Generate negative pairs (different students)
    for i in range(len(students)):
        for j in range(i + 1, len(students)): 
            student1_path = os.path.join(image_dir, students[i])
            student2_path = os.path.join(image_dir, students[j])

            if not os.path.isdir(student1_path) or not os.path.isdir(student2_path):
                continue
            student1_images = os.listdir(student1_path)
            student2_images = os.listdir(student2_path)
            img1 = f"{image_dir}/{students[i]}/{random.choice(student1_images)}"
            img2 = f"{image_dir}/{students[j]}/{random.choice(student2_images)}"
            pairs.append([preprocess_image(img1), preprocess_image(img2)])
            labels.append(0)

    return np.array(pairs), np.array(labels)


image_dir = "dataset"
pairs, labels = create_image_pairs(image_dir)



<h1>Train Model</h1>


In [ ]:
# Convert to tensors
pairs = tf.convert_to_tensor(pairs, dtype=tf.float32)
labels = tf.convert_to_tensor(labels, dtype=tf.float32)


# Define base network
def create_base_network(input_shape=(128, 128, 1)):
    input_layer = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (7, 7), activation="relu", padding="same")(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (5, 5), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)
    return Model(input_layer, x)


# Create Siamese network
def create_siamese_network(input_shape=(128, 128, 1)):
    base_network = create_base_network(input_shape)
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    # distance = layers.Lambda(lambda tensors: tf.math.abs(tensors[0] - tensors[1]))(
    #     [processed_a, processed_b]
    # )

    l1_distance = layers.Subtract()([processed_a, processed_b])
    l1_distance = layers.Activation('relu')(l1_distance)
    # l1_distance = layers.Lambda(lambda x: tf.math.abs(x), output_shape=lambda x: x)(l1_distance) 
    
    output = layers.Dense(1, activation="sigmoid")(l1_distance)
    return Model([input_a, input_b], output)


# Instantiate and compile the model
siamese_model = create_siamese_network()
siamese_model.compile(
    optimizer=Adam(0.0001), loss=BinaryCrossentropy(), metrics=["accuracy"]
)

# Train the model
history = siamese_model.fit(
    [pairs[:, 0], pairs[:, 1]], labels, batch_size=16, epochs=20, validation_split=0.2
)

# create model dir if not created
os.makedirs("C:/Users/USER/Documents/GitHub/Facial-Recognition-Attendance-System/model", exist_ok=True) 

# Save the model
siamese_model.save("model/siamese_model.keras")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
siamese_model = load_model("model/siamese_model.keras", safe_mode=False)

# Static image path
static_image_path = "temp-test-image.jpg" 
static_image = preprocess_image(static_image_path)

# Function to test the static image against all images in the dataset
def test_static_image_against_dataset(static_image, dataset_dir, threshold=0.5):
    students = os.listdir(dataset_dir)
    results = {}

    for student in students:
        student_path = os.path.join(dataset_dir, student)
        if not os.path.isdir(student_path):
            continue

        student_images = os.listdir(student_path)
        match_count = 0

        # Compare the static image to each image in this student's folder
        for image_name in student_images:
            image_path = os.path.join(student_path, image_name)
            test_image = preprocess_image(image_path)
            
            # Predict similarity
            prediction = siamese_model.predict([static_image, test_image])
            similarity_score = prediction[0][0]
            
            # Check if similarity score is above threshold
            if similarity_score > threshold:
                match_count += 1

        # Save results for this student
        results[student] = match_count

    # Print match results
    for student, count in results.items():
        print(f"Matches for {student}: {count}/{len(os.listdir(os.path.join(dataset_dir, student)))} images")

# Run the test
test_static_image_against_dataset(static_image, "dataset")


In [ ]:
# testing purposes

static_image_path = "dataset\Muhammad Khan\image2.jpg" 
static_image = preprocess_image(static_image_path)

test_image_path = "dataset\Omer Khan\image2.jpg"

# Predict similarity
prediction = siamese_model.predict([static_image, test_image])
prediction[0][0]


<h1>Model using website code</h1>


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import numpy as np
import os
import random
import cv2

# 1. Embedding Model Definition (Keep as is, working correctly)
def build_embedding_model(input_shape=(128, 128, 1), embedding_dim=48):
    inputs = Input(input_shape)
    x = layers.Conv2D(64, (2, 2), padding="same", activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(64, (2, 2), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Dropout(0.3)(x)
    pooled_output = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(embedding_dim)(pooled_output)
    return Model(inputs, outputs)

# 2. Improved Contrastive Loss
@tf.keras.utils.register_keras_serializable(name="contrastive_loss")
def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = tf.cast(y_true, y_pred.dtype)
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# 3. Improved Siamese Network
def build_siamese_network(input_shape):
    base_network = create_base_network(input_shape)
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
    outputs = Dense(1, activation="sigmoid")(distance)

    return Model(inputs=[input_a, input_b], outputs=outputs)

def create_base_network(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation="relu")(inputs)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation="relu")(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation="relu")(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation="relu")(x)
    x = Flatten()(x)
    x = Dense(4096, activation="sigmoid")(x)
    return Model(inputs, x)

# 4. Improved Image Loading and Preprocessing
def preprocess_image(image_path, target_size=(128, 128)):
    """
    Preprocesses an image for the Siamese network.
    """
    if isinstance(image_path, str):
        # Load from file
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    else:
        # Already a numpy array
        image = image_path if len(image_path.shape) == 2 else cv2.cvtColor(image_path, cv2.COLOR_BGR2GRAY)

    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=-1)
    return np.expand_dims(image, axis=0)

def load_images_from_folder(folder_path, target_size=(128, 128)):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = preprocess_image(img_path, target_size)[0]  # Remove the batch dimension
            images.append(img)
    return np.array(images)

# 5. Improved Pair Creation
def create_pairs(data_dir):
    pairs = []
    labels = []
    student_folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

    for student in student_folders:
        student_path = os.path.join(data_dir, student)
        images = load_images_from_folder(student_path)

        if len(images) < 2:
            continue

        # Positive pairs
        for i in range(len(images) - 1):
            pairs.append([images[i], images[i + 1]])
            labels.append(1)

        # Negative pairs
        other_students = [s for s in student_folders if s != student]
        for _ in range(len(images)):
            neg_student = random.choice(other_students)
            neg_path = os.path.join(data_dir, neg_student)
            neg_images = load_images_from_folder(neg_path)
            if len(neg_images) > 0:
                pairs.append([images[random.randint(0, len(images)-1)],
                            neg_images[random.randint(0, len(neg_images)-1)]])
                labels.append(0)

    return [np.array([p[0] for p in pairs]), np.array([p[1] for p in pairs])], np.array(labels)

# 6. Improved Testing Function
def test_facial_recognition(model, test_image_path, dataset_path):
    """
    Tests a single image against a dataset of known faces.
    Returns the best matching student and confidence scores.
    """
    test_image = preprocess_image(test_image_path)
    results = []

    for student in os.listdir(dataset_path):
        student_path = os.path.join(dataset_path, student)
        if not os.path.isdir(student_path):
            continue

        student_scores = []
        student_images = load_images_from_folder(student_path)

        for ref_image in student_images:
            ref_image = np.expand_dims(ref_image, axis=0)
            similarity = model.predict([test_image, ref_image], verbose=0)[0][0]
            student_scores.append(similarity)

        if student_scores:
            avg_score = np.mean(student_scores)
            max_score = np.max(student_scores)
            results.append({
                'student': student,
                'average_score': avg_score,
                'max_score': max_score,
                'matches': sum(1 for score in student_scores if score > 0.5)  # This can be adjusted
            })

    # Sort by average score
    results.sort(key=lambda x: x['average_score'], reverse=True)
    return results

# Training script
def train_siamese_network(data_dir, input_shape=(128, 128, 1), epochs=50, batch_size=32):
    # Create and compile model
    siamese_model = build_siamese_network(input_shape)
    siamese_model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=contrastive_loss,
        metrics=['accuracy']
    )

    # Create training pairs
    train_pairs, train_labels = create_pairs(data_dir)

    # Train the model
    history = siamese_model.fit(
        [train_pairs[0], train_pairs[1]],
        train_labels,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2
    )

    return siamese_model, history

# Example usage
if __name__ == "__main__":
    # Training
    data_dir = "/content/drive/MyDrive/Colab Notebooks/Facial-Recognition-Attendance-System/dataset"
    model, history = train_siamese_network(data_dir)
    model.save("/content/drive/MyDrive/Colab Notebooks/Facial-Recognition-Attendance-System/model/siamese_model.keras")

    # Testing
    test_image_path = "/content/drive/MyDrive/Colab Notebooks/Facial-Recognition-Attendance-System/dataset/Muhammad Khan/image10.jpg"
    results = test_facial_recognition(model, test_image_path, data_dir)

    # Print results
    for result in results:
        print(f"Student: {result['student']}")
        print(f"Average similarity score: {result['average_score']:.3f}")
        print(f"Maximum similarity score: {result['max_score']:.3f}")
        print(f"Number of matches: {result['matches']}")
        print("-" * 50)